# Federal University of Ceará
# Teleinformatics Departament
# Graduate Program in Teleinformatics Engeneering
## TIP8419 - Tensor Algebra
## Homework 8 - Multidimensional Least-Squares Kronecker Factorization (MLS-KronF)
### Report and Simulation results

- Ezequias Márcio - 497779

To run this notebook properly, it is necessary Python3 installed alongside with the packages listed below:

- `numpy 1.17.2`
- `scipy 1.4.1`
- `tdqm 4.36.1`
- `bokeh 1.3.4`

Make sure that the files `tensoralg.py` and `ta_simulations.py` are in the same directory as this notebook. In this files, it can be found the tensor algebra module functions and the code listings of the simulations.

In [1]:
# Importing the simulation module:
from ta_simulations import *
np.set_printoptions(4, linewidth=175)
output_notebook()
# Loading the .m file given as a python dictionary:
kronf_data = loadmat('files/kronf_matrix_3D.mat')

Loading BokehJS ...

### Problem 1

On practice 4 we implement the LS-KronF(Least Square Kronecker Factorization) algorithm, now we will go to implement the MLS-KronF(Multidimensional Least Square Kronecker Factorization) algorithm. Then, Let $\mathbf{X} \approx \mathbf{A}^{(1)} \otimes \mathbf{A}^{(2)} \otimes ... \otimes \mathbf{A}^{(N)} \in \mathbb{C}^{I_{1} I_{2} ... I_{N} \times J_{1} J_{2} ... J_{N}}$ be a matrix composed by Kronecker product of N matrices $\mathbf{A}^{(n)} \in \mathbb{C}^{I_{n} \times J_{n}}$ , with n = 1, 2, . . . , N . For N = 3 and $I_{n}$ and $J_{n}$ arbitrary implement the MLS-KRF for that estimate $\mathbf{A}^{(1)}, \mathbf{A}^{(2)}$ and $\mathbf{A}^{(3)}$ by solving the following problem.

\begin{equation}
    (\hat{\mathbf{A}}^{(1)}, \hat{\mathbf{A}}^{(2)}, \hat{\mathbf{A}}^{(3)}) = \underset{\mathbf{A}, \mathbf{B}}{\text{min}} || \mathbf{X} - \mathbf{A}^{(1)} \otimes \mathbf{A}^{(2)} \otimes \mathbf{A}^{(3)} ||^{2}_{F}
\end{equation}

Compare the estimate matrices $\hat{\mathbf{A}}^{(1)}, \hat{\mathbf{A}}^{(2)}$ and $\hat{\mathbf{A}}^{(3)}$ with the original ones. What can you
conclude? Explain the results.

### Solution:

In the cell below, the matrices $\mathbf{A}^{(1)} \in \mathbb{C}^{4×2}$, $\mathbf{A}^{(2)} \in \mathbb{C}^{2×3}$ and $\mathbf{A}^{(3)} \in \mathbb{C}^{2×2}$ are randomly generated and is calculated the squared error between these matrices and their estimates for comparison. The implemented routine for this factorization is in the file `tensoralg.py` as `lskronf_3d`. Note that the functions implemented only consider the $N=3$ matrices case.

In [2]:
# Testing the Multidimensional Least-Squares Kronecker Factorization:
shapes = [(4, 2), (2, 3), (2, 2)] # Shapes for each matrix A
A = [rand(shape[0], shape[1]*2).view(np.complex_) for shape in shapes] # Generating random matrices: A1, A2, A3
X = tensoralg.kron(*A); print(f'Matrix X {X.shape}') # Construct matrix X
# Estimating matrices A_hat and X_hat:
A_hat = tensoralg.lskronf_3d(X, shapes); X_hat = tensoralg.kron(*A_hat)
# Calculating the Normalized Mean Squared Error:
nmse_A = [norm_mse(A[n], A_hat[n]) for n in range(len(shapes))]; nmse_X = norm_mse(X, X_hat)
print(f'''Normalized Mean Squared Errors:\n- Matrix A: {nmse_A}\n- Matrix X: {nmse_X}''')
print('\nUsing HOOI:')
# Estimating matrices A_hat and X_hat:
A_hat = tensoralg.lskronf_3d(X, shapes, True); X_hat = tensoralg.kron(*A_hat)
# Calculating the Normalized Mean Squared Error:
nmse_A = [norm_mse(A[n], A_hat[n]) for n in range(len(shapes))]; nmse_X = norm_mse(X, X_hat)
print(f'''Normalized Mean Squared Errors:\n- Matrix A: {nmse_A}\n- Matrix X: {nmse_X}''')

Matrix X (16, 12)
Normalized Mean Squared Errors:
- Matrix A: [2.8604778804101922, 2.6491179163266563, 3.5674694254973907]
- Matrix X: 1.257372497354007e-31

Using HOOI:
Normalized Mean Squared Errors:
- Matrix A: [2.8604778804101927, 2.6491179163266563, 3.5674694254973898]
- Matrix X: 1.3160009080335287e-31


As can be seen by the result above, the error between the estimated matrices $\hat{\mathbf{A}}^{(n)}$ and the oringinal ones is high. However, the error between the reconstructed version of $\mathbf{X}$ $(\hat{\mathbf{A}}^{(1)}\otimes\hat{\mathbf{A}}^{(2)} \otimes \hat{\mathbf{A}}^{(3)})$ and the original is minimized by the MLSKRONF algorithm.

Again, as in the LSKRONF case, because only the first $n$-th singular value and vector are chosen for the truncated HOSVD/HOOI, the estimated matrices are not close to the original ones, that is evidenced by the high error values above. Also, for the MLSKRONF using the HOOI for the estimation the obtained result is according to the expected.

- Validating the results:

Using the provided matrices in `kronf_matrix_3D.mat`, it can be seen below that the implemented algorithm is working as expected, presenting a small error for the reconstructed matrix.

In [3]:
# Validating the results with the matrices provided:
a_test = kronf_data['A']; b_test = kronf_data['B']; c_test = kronf_data['C']; x_test = kronf_data['X']
print(f'Matrix X {x_test.shape}\nMatrix A {a_test.shape}\nMatrix B {b_test.shape}\nMatrix C {c_test.shape}')
# Estimating matrices A, B and C:
matrices = [a_test, b_test, c_test]; test_shapes = [m.shape for m in matrices]
m_hat = tensoralg.lskronf_3d(x_test, test_shapes); x_hat = tensoralg.kron(*m_hat)
# Calculating the squared error
# Calculating the Normalized Mean Squared Error:
nmse_test = [norm_mse(matrices[n], m_hat[n]) for n in range(len(test_shapes))]; nmse_x_test = norm_mse(x_test, x_hat)
print(f'''Normalized Mean Squared Errors:\n- Matrices A, B, C: {nmse_test}\n- Matrix X: {nmse_x_test}''')

Matrix X (96, 30)
Matrix A (4, 3)
Matrix B (4, 2)
Matrix C (6, 5)
Normalized Mean Squared Errors:
- Matrices A, B, C: [3.9451930075709254, 3.782147264496461, 2.017214166304976]
- Matrix X: 7.751981180553917e-31


### Problem 2: 

Assuming 1000 Monte Carlo experiments, generate  $\mathbf{X_{0}} = \mathbf{A} \otimes \mathbf{B} \otimes \mathbf{C} \in \mathbb{C}^{I_{1} I_{2} I_{3} \times J_{1} J_{2} J_{3}}$, for randomly chosen $\mathbf{A} \in \mathbb{C}^{I_{1} \times J_{1}}$, $\mathbf{B} \in \mathbb{C}^{I_{2} \times J_{2}}$ and $\mathbf{C} \in \mathbb{C}^{I_{3} \times J_{3}}$, whose elements are drawn from a normal distribution. Let $\mathbf{X} = \mathbf{X_{0}} + \alpha \mathbf{V}$ be a noisy version of $\mathbf{X}$ where $\mathbf{V}$ is the additive noise term, whose elements are drawn from a normal distribution. The parameter $\alpha$ controls the power (variance) of the noise term, and is defined as a function of the signal to noise ratio (SNR), in dB, as follows:

\begin{equation}
    \text{SNR}_{dB} = 10 \log_{10} \frac{||\mathbf{X_{0}}||^{2}_{F}}{||\alpha \mathbf{V}||^{2}_{F}}
\end{equation}

Assuming the SNR range $[0, 5, 10, 15, 20, 25, 30]$ dB, find the estimates $\hat{\mathbf{A}}$, $\hat{\mathbf{B}}$ and $\hat{\mathbf{C}}$ obtained with the MLS-KRONF algorithm for the configurations $I_{1} = J_{1}  = 2$, $I_{2} =J_{2}= 3$ and $I_{3} =J_{3}= 4$. 

Let us define the normalized mean square error (NMSE) measure as follows:

\begin{equation}
    \text{NMSE}(\mathbf{X_{0}}) = \frac{1}{1000} \sum^{1000}_{i = 1} \frac{||\hat{ \mathbf{X}}_{0} (i) - \mathbf{X_{0}}(i)||^{2}_{F}}{||\mathbf{X_{0}}(i)||^{2}_{F}}
\end{equation}

where $\mathbf{X_{0}}(i)$ and $\hat{\mathbf{X}}_{0}(i)$ represent the original data matrix and the reconstructed one at the ith experiment, respectively. For each SNR value and configuration, plot the NMSE vs. SNR curve. Discuss the obtained results.

### Solution:

The simulation results are generated in the cell below. First, for the 1000 realizations, the matrix $\mathbf{X}_{0}(i)$ is generated and then, for each value of SNR, it is added Gaussian noise to the matrix tha will be the input of the MLSKRONF algorithm. After that, the estimated matrices $\hat{\mathbf{A}}, \hat{\mathbf{B}}$ and $\hat{\mathbf{C}}$ are used to build the estimative $\hat{\mathbf{X}}_{0}(i)$. Lastly, the NMSE between the estimated matrix and the original is sorted. This process is implemented in the function `run_simulation_lskron3d`.

In [4]:
shapes=[(2, 2), (3, 3), (4, 4)]  # Shapes I1, J1; I2, J2 and I3; J3
snr = np.linspace(0, 30, 7)  # SNR values
mc_realizations = 1000       # Monte Carlo Realizations
#Generating data for the two cases: HOSVD and HOOI:
result1 = run_simulation_lskron3d(snr, mc_realizations, shapes)
result2 = run_simulation_lskron3d(snr, mc_realizations, shapes, True)

The NMSE vs. SNR curve is presented in a log-scale plot below.

As can be seen in the figure below, with the increase in SNR, the estimative is more accurate because the noise power is decreasing. Also, with respect to the case where the HOOI algorithm is applied the performance of the MLSKRONF algorithm is practically the same.

In [6]:
plot_results(snr, result1, result2, 'using HOSVD', 'using HOOI', 'MLSKRONF')